# Further Production Optimisation

Removal of low value features to simplify model for production deployment

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    recall_score,
    roc_curve,
    roc_auc_score,
)
import shap

from sklearn.model_selection import (
    GroupShuffleSplit,
)
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.metrics import AUC
import pickle


## Pipeline Configuration

In [2]:
random_seed = 1337  # Random seed to ensure reproducibility
output_path = "output/tensorflow_optimisation/"
cmap = "seismic"  # Colormap for SHAP plots use "seismic" for full cohort and "berlin" for biochem remission cohort
file_prefix = "tensorflow_optimisation"
# file_prefix = "biochem_remission"

## Data Loading

In [3]:
df = pd.read_csv("working_data/all_ibd_ml_input.csv")
# df = df[df["aggregate_disease_activity_Biochemical remission"] == 1] # Uncomment to run biochem remission pipeline

## Further Data Pre-Processing

In [5]:
# convert categorical columns to numerical
df["sex"] = df["sex"].map({"Male": 1, "Female": 0})
df["fatigue_outcome"] = df["fatigue_outcome"].map({"fatigue": 1, "no_fatigue": 0})

In [6]:
# These columns are not features we want to use in the model
# Aggregate disease activity in some ways is a reflection of the other raw variables
columns_to_drop = [
    "aggregate_disease_activity_Active",
    "aggregate_disease_activity_Biochemical remission",
    "aggregate_disease_activity_Remission",
    "season_no_data",
    "study",
    "redcap_event_name_timepoint_1",
    "redcap_event_name_timepoint_2",
    "redcap_event_name_timepoint_3",
    "redcap_event_name_timepoint_4",
    "redcap_event_name_timepoint_5",
]

df.drop(columns=columns_to_drop, inplace=True)

In [7]:
# This column is dropped as all the values are 0.
columns_to_drop = [
    "baseline_eims_pyoderma_gangrenosum",
]

df.drop(columns=columns_to_drop, inplace=True)

In [8]:
# Removed to simplify deployment
columns_to_drop = [
    "has_active_symptoms",
]

df.drop(columns=columns_to_drop, inplace=True)

In [9]:
# We will drop features that are not highly weighted to simplify the input model

columns_to_drop = [
    "baseline_eims_arthralgia_arthritis",
    "baseline_eims_ankylosing_spondylitis",
    "baseline_eims_erythema_nodosum",
    "baseline_eims_uveitis",
    "baseline_eims_scleritis_episclerities",
    "is_smoker_smokeryn1",
    "study_group_name_Await Dx",
    "ifx_drug_level",
    "ada_drug_level",
    "ifx_drug_level_present",
    "ada_drug_level_present",
    "ifx_antibody_present",
    "ada_antibody_present",
    "haematocrit",
]

df.drop(columns=columns_to_drop, inplace=True)

## Create Train and Test Datasets

GroupShuffleSplit used to ensure same participant only appears in either train or test set.

In [10]:
# Create Train Validate and Test Datasets

# First split into train and temp 70% train, 30% temp which will be split 50:50 into 15% val and 15% test

# GroupShuffleSplit
splitter = GroupShuffleSplit(test_size=0.36, n_splits=1, random_state=random_seed)

# Perform the split
for train_idx, test_idx in splitter.split(df, groups=df["study_id"]):
    train_data = df.iloc[train_idx]
    temp_data = df.iloc[test_idx]

# Drop 'study_id' from X_train and X_test as it's not a feature
X_train = train_data.drop(columns=["fatigue_outcome", "study_id"])
y_train = train_data["fatigue_outcome"]

groups = train_data["study_id"]  # Group variable for GroupKFold cross-validation

temp_data_splitter = GroupShuffleSplit(
    test_size=0.56, n_splits=1, random_state=random_seed
)

# Perform the split
for val_idx, test_idx in temp_data_splitter.split(
    temp_data, groups=temp_data["study_id"]
):
    val_data = df.iloc[val_idx]
    test_data = df.iloc[test_idx]

X_val = val_data.drop(columns=["fatigue_outcome", "study_id"])
y_val = val_data["fatigue_outcome"]

X_test = test_data.drop(columns=["fatigue_outcome", "study_id"])
y_test = test_data["fatigue_outcome"]


In [ ]:
print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)
print("Test shape:", X_test.shape)

In [12]:
numerical_features = [
    "age",
    "height",
    "weight",
    "bmi",
    "age_at_diagnosis",
    "albumin",
    "crp",
    "haemoglobin",
    "red_cell_count",
    # "haematocrit",
    "white_cell_count",
    "neutrophils",
    "lymphocytes",
    "monocytes",
    "eosinophils",
    "basophils",
    "platelets",
    "urea",
    "creatinine",
    "sodium",
    "potassium",
    "calprotectin",
    # "ada_drug_level",
    # "ifx_drug_level",
    "diagnosis_year",
    "disease_duration_weeks",
]

X_unified = pd.concat([X_train, X_val, X_test])
unified_scaler = StandardScaler()
unified_scaler.fit(X_unified[numerical_features])

X_train[numerical_features] = unified_scaler.transform(X_train[numerical_features])

X_test[numerical_features] = unified_scaler.transform(X_test[numerical_features])

X_val[numerical_features] = unified_scaler.transform(X_val[numerical_features])


In [ ]:
from joblib import dump

dump(unified_scaler, output_path + "export/scaler.pkl")

In [14]:
X_train.to_csv(output_path + "export/X_train.csv", index=False)
X_test.to_csv(output_path + "export/X_test.csv", index=False)

## Deep Learning with TensorFlow


### Model Definition

In [15]:
model = keras.Sequential(
    [
        layers.Dense(384, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(352, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(1, activation="sigmoid"),
    ]
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="auc", patience=10, restore_best_weights=True
)

### Compiling the Model

In [16]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=[AUC()])

In [ ]:
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping_callback],
)

In [ ]:
model.summary()

In [19]:
model.save(f"{output_path}export/fatigue_model.keras")

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
auc_values = history_dict["auc"]
val_auc_values = history_dict["val_auc"]
epochs = range(1, len(auc_values) + 1)
plt.plot(epochs, auc_values, "bo", label="Training AUC")
plt.plot(epochs, val_auc_values, "b", label="Validation AUC")
plt.title("Training and validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()

save_path = output_path + file_prefix + "_training_vs_validation_loss.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

save_path = output_path + file_prefix + "_training_vs_validation_loss.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
plt.clf()
acc = history_dict["auc"]
val_acc = history_dict["val_auc"]
plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend()
save_path = output_path + file_prefix + "_training_vs_validation_auc.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
results = model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred)

y_classes = np.where(y_pred > 0.5, 1, 0)

tn, fp, fn, tp = confusion_matrix(y_test, y_classes).ravel()

# Calculate metrics
accuracy = accuracy_score(y_test, y_classes)
sensitivity = recall_score(y_test, y_classes)  # TPR
specificity = tn / (tn + fp)  # TN

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("AUC:", test_auc)

In [26]:
np.savetxt(output_path + "dnn_fpr.txt", fpr)
np.savetxt(output_path + "dnn_tpr.txt", tpr)

with open(output_path + "dnn_auc.txt", "w") as f:
    f.write(str(test_auc))

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(
    fpr,
    tpr,
    label=f"AUC = {test_auc:.4f}",
)

# Add baseline and plot details
plt.plot([0, 1], [0, 1], "k--", label="Chance (AUC = 0.50)")
plt.xlabel("False Positive Rate", fontsize=14)
plt.ylabel("True Positive Rate", fontsize=14)
plt.title("Performance of Keras DNN on Test Set", fontsize=16, fontweight="bold")
plt.legend(loc="lower right")

# Remove the top and right spines
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Define the absolute path
save_path = f"{output_path}{file_prefix}_roc_curves.png"

# Save the plot to the specified path
plt.savefig(save_path, dpi=300, bbox_inches="tight")

# Display the plot
plt.show()

## View Tensorboard Output

To launch tensorboard run the following in terminal:

```bash
tensorboard --logdir logs/
```

## SHAP Analysis on Keras DNN Model

In [31]:
explainer = shap.KernelExplainer(model, X_train[:50])

In [33]:
test_df = X_test[:1]

In [ ]:
shap_values = explainer.shap_values(test_df)

In [ ]:
print(shap_values)

In [ ]:
shap_values.shape

In [ ]:
model.predict(test_df)

In [ ]:
shap.force_plot(
    explainer.expected_value[0],
    shap_values[:1, :, 0],
    X_test[:1],
    matplotlib=True,
    contribution_threshold=0.05,
    text_rotation=30,
    show=False,
)

In [35]:
with open(output_path + "export/shap_explainer.pkl", "wb") as f:
    pickle.dump(explainer, f)

In [36]:
with open(f"{output_path}export/shap_explainer.pkl", "rb") as f:
    new_explainer = pickle.load(f)

In [ ]:
shap_values_test = new_explainer.shap_values(X_test.iloc[1])

print(shap_values_test)

In [ ]:
X_test.iloc[1]

In [ ]:
shap.force_plot(
    explainer.expected_value[0],
    shap_values_test[:, 0],
    X_test.iloc[1],
    matplotlib=True,
    contribution_threshold=0.05,
    text_rotation=30,
    show=False,
)

In [ ]:
shap_values_class_1 = shap_values[:, :, 0]
shap.summary_plot(
    shap_values_class_1,
    X_test,
    feature_names=X_test.columns,
    show=False,
    cmap=cmap,
)

# plt.title("Keras DNN Classifier", fontsize=20, pad=20, loc="left")

save_path = f"{output_path}shap_keras_dnn.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
shap.summary_plot(
    shap_values_class_1,
    X_test,
    feature_names=X_test.columns,
    show=False,
    cmap=cmap,
    plot_type="bar",
    max_display=85,
)


plt.title("Keras DNN Classifier - Top 10 Features", fontsize=20, pad=20, loc="left")

save_path = f"{output_path}shap_keras_dnn_barplot.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
shap.initjs()

In [28]:
X_test_reverted = X_test

In [29]:
X_test_reverted[numerical_features] = unified_scaler.inverse_transform(
    X_test[numerical_features]
)

In [30]:
X_test_reverted[numerical_features] = X_test_reverted[numerical_features].round(2)

In [ ]:
explainer.expected_value

In [ ]:
for i in range(10):
    shap.force_plot(
        explainer.expected_value[0],
        shap_values_class_1[i],
        X_test_reverted.iloc[i],
        matplotlib=True,
        contribution_threshold=0.05,
        text_rotation=30,
        show=False,
    )

    save_path = f"{output_path}forceplots/keras_dnn_forceplot_{i}.png"
    plt.savefig(save_path, dpi=150, bbox_inches="tight")
    plt.show()